In [ ]:
"""
Created on Thu Jan 16 14:55 2023

Trying to do overview plots like Pierre

Author: @claraburgard
"""

In [1]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import cmocean

import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
import seaborn as sns
import cartopy
import matplotlib.colors as colors
from matplotlib.colors import Normalize
from cartopy.feature import LAND

from basal_melt_NEMO.constants import *
import basal_melt_NEMO.figure_functions as figf
import basal_melt_NEMO.useful_functions as uf

In [2]:
sns.set_context('paper')

In [3]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ IN DATA

In [4]:
inputpath_closed='/data/cdelaver/n42tm21/'
inputpath_open='/data/cdelaver/n42openc/'
clara_path='/data/cburgard/CASIMIR_SIMU/interim/XR_PROCESSED/'
plot_path = '/data/cburgard/PLOTS/first_plots/'

In [5]:
### FILE T PREPARATION

In [6]:
file_open_list = []
file_closed_list = []

for yy in range(9):
    yy1=yy+1
    file_open = xr.open_mfdataset(inputpath_open + 'n42openc_00'+str(yy)+'10101_00'+str(yy1)+'01231_1Y_grid_T.nc')
    file_closed = xr.open_mfdataset(inputpath_closed + 'n42tm21_00'+str(yy)+'10101_00'+str(yy1)+'01231_1Y_grid_T.nc')
    file_open_list.append(file_open)
    file_closed_list.append(file_closed)
file_T_open_all = xr.concat(file_open_list, dim='time_counter')
file_T_closed_all = xr.concat(file_closed_list, dim='time_counter')


In [7]:
file_T_open_all = file_T_open_all.rename({'time_counter':'time'}) 
file_T_open_all['time'] = file_T_open_all['time'].dt.year
file_T_open_all = file_T_open_all.where(file_T_open_all['so'] > 0)

file_T_closed_all = file_T_closed_all.rename({'time_counter':'time'}) 
file_T_closed_all['time'] = file_T_closed_all['time'].dt.year

In [8]:
mask_land_closed = file_T_closed_all['so'].isel(time=0,deptht=0)
mask_land_open = file_T_open_all['so'].isel(time=0)

In [9]:
cellarea_open = file_T_open_all['area'].isel(time=0).where(mask_land_open > 0).drop('time')
cellarea_closed = file_T_closed_all['area'].isel(time=0).where(mask_land_closed > 0).drop('time')

In [10]:
### FILE U PREPARATION

In [11]:
file_open_list = []
file_closed_list = []

for yy in range(9):
    yy1=yy+1

    file_open = xr.open_mfdataset(inputpath_open + 'n42openc_00'+str(yy)+'10101_00'+str(yy1)+'01231_1Y_grid_U.nc')
    file_closed = xr.open_mfdataset(inputpath_closed + 'n42tm21_00'+str(yy)+'10101_00'+str(yy1)+'01231_1Y_grid_U.nc')
    file_open_list.append(file_open)
    file_closed_list.append(file_closed)
    
file_U_open_all = xr.concat(file_open_list, dim='time_counter')
file_U_closed_all = xr.concat(file_closed_list, dim='time_counter')

file_U_open_all = file_U_open_all.rename({'time_counter':'time'}) 
file_U_open_all['time'] = file_U_open_all['time'].dt.year

file_U_closed_all = file_U_closed_all.rename({'time_counter':'time'}) 
file_U_closed_all['time'] = file_U_closed_all['time'].dt.year

In [12]:
u_vertsum_open = (file_U_open_all['uocetr_eff'] * (mask_land_open.rename({'deptht':'depthu'}) > 0)).sum('depthu')
u_vertsum_closed = (file_U_closed_all['uocetr_eff'] * (mask_land_closed.rename({'deptht':'depthu'}) > 0)).sum('depthu')

In [13]:
### VAR LIST

In [14]:
var_list = ['global_temp','wed_gyre','ross_gyre','ACC',
            'FRIS_melt','Ross_melt','LarsenC_melt','total_melt',
            'mar_sie_arc','sep_sie_arc','feb_sie_ant','sep_sie_ant',
            'Sbot_WWED','Sbot_EWED','Sbot_WROSS','Sbot_EROSS','Sbot_AMU',
            'Tbot_WWED','Tbot_EWED','Tbot_WROSS','Tbot_EROSS','Tbot_AMU']



In [15]:
var_obs_mean = xr.DataArray(data=np.array([15.0, 56.0, 20.0, 136.7, 
               155.4, 47.7, 20.7, 1500,
               15.3, 6.3, 3.3, 19.8, 
               34.9, np.nan, 35.0, np.nan, np.nan, 
               np.nan, -1.95, -1.9, -1.7, np.nan]), dims='var').assign_coords({'var': var_list})

In [16]:
var_obs_std = xr.DataArray(data=np.array([0, 8.0, 5.0, 7.8, 
              45.0, 34.0, 67.0, 237,
              1.0, 1.0, 0.5, 0.5, 
              0.0, np.nan, 0.0, np.nan, np.nan,
              np.nan, 0.2, 0.4, 0.4, np.nan]), dims='var').assign_coords({'var': var_list})

In [17]:
var_to_plot = xr.Dataset()

In [19]:
### GLOBAL TEMP
var_to_plot['global_temp'] = xr.concat([uf.weighted_mean(file_T_open_all['thetao'].isel(deptht=0),['x','y'],cellarea_open.isel(deptht=0)).assign_coords({'cav':'open'}),
                                        uf.weighted_mean(file_T_closed_all['thetao'].isel(deptht=0),['x','y'],cellarea_closed).assign_coords({'cav':'closed'})], 
                                       dim='cav').load()

In [20]:
### ACC
var_to_plot['ACC'] = xr.concat([u_vertsum_open.sel(x=220,y=range(79,107)).sum('y').assign_coords({'cav':'open'}),
                                u_vertsum_closed.sel(x=220,y=range(79,107)).sum('y').assign_coords({'cav':'closed'})], dim='cav')
var_to_plot['ACC'] = var_to_plot['ACC']/10**6

In [21]:
### GYRES
map_var = cellarea_closed

uocetr_Wed_open = u_vertsum_open.where(uf.in_range(map_var.nav_lat,[-66.50,-60.40]) & uf.in_range(map_var.nav_lon,[-31.25,37.50]), drop=True)
uocetr_Wed_closed = u_vertsum_closed.where(uf.in_range(map_var.nav_lat,[-66.50,-60.40]) & uf.in_range(map_var.nav_lon,[-31.25,37.50]), drop=True)

uocetr_Ross_open = u_vertsum_open.where(uf.in_range(map_var.nav_lat,[-72.650,-61.600]) & ((map_var.nav_lon <= -135.75) | (map_var.nav_lon >= 360-168.500)), drop=True)
uocetr_Ross_closed = u_vertsum_closed.where(uf.in_range(map_var.nav_lat,[-72.650,-61.600]) & ((map_var.nav_lon <= -135.75) | (map_var.nav_lon >= 360-168.500)), drop=True)


In [22]:
var_to_plot['ross_gyre'] = xr.concat([uocetr_Ross_open.cumsum('y').max(['y','x']).assign_coords({'cav':'open'}), uocetr_Ross_closed.cumsum('y').max(['y','x']).assign_coords({'cav':'closed'})], dim='cav')/10**6
var_to_plot['wed_gyre'] = xr.concat([uocetr_Wed_open.cumsum('y').max(['y','x']).assign_coords({'cav':'open'}), uocetr_Wed_closed.cumsum('y').max(['y','x']).assign_coords({'cav':'closed'})], dim='cav')/10**6


In [23]:
### ICE SHELF MELT
T_grid_vars_open = xr.open_dataset('/data/cburgard/CASIMIR_SIMU/interim/CDO_PROCESSED/n42openc/n42openc_0-100_1Y_grid_isfvars.nc')

lon = T_grid_vars_open.nav_lon
lat = T_grid_vars_open.nav_lat

T_grid_vars_open = T_grid_vars_open.rename({'time_counter':'time'}) 
T_grid_vars_open['time'] = T_grid_vars_open['time'].dt.year

mask_FRIS = (lon <= -30) & (lon >= -80.) & (lat >= -84.) & (lat <= -74.)
mask_Ross = ((lon <= -120) | (lon >= 150)) & (lat >= -87.) & (lat <= -65.)
mask_LarsenC = (lon <= -58) & (lon >= -66) & (lat >= -71.) & (lat <= -64.)

In [24]:
melt_all = (T_grid_vars_open['iceshelf_cav'] * cellarea_open.max('deptht') * yearinsec *10**(-12))
FRIS_melt= melt_all.where(mask_FRIS).sum(['x','y'])
Ross_melt = melt_all.where(mask_Ross).sum(['x','y'])
LarsenC_melt = melt_all.where(mask_LarsenC).sum(['x','y'])

var_to_plot['FRIS_melt'] = xr.concat([FRIS_melt.assign_coords({'cav':'open'}), FRIS_melt.assign_coords({'cav':'closed'}) * np.nan], dim='cav')
var_to_plot['Ross_melt'] = xr.concat([Ross_melt.assign_coords({'cav':'open'}), Ross_melt.assign_coords({'cav':'closed'}) * np.nan], dim='cav')
var_to_plot['LarsenC_melt'] = xr.concat([LarsenC_melt.assign_coords({'cav':'open'}), LarsenC_melt.assign_coords({'cav':'closed'}) * np.nan], dim='cav')

var_to_plot['total_melt'] = var_to_plot['FRIS_melt'] + var_to_plot['Ross_melt'] + var_to_plot['LarsenC_melt'] + (T_grid_vars_open['iceshelf'] * cellarea_open.max('deptht') * yearinsec * 10**(-12)).sum(['x','y'])


In [25]:
### SEA ICE

file_open_list = []
file_closed_list = []

for yy in range(9):

    file_open = xr.open_mfdataset(inputpath_open+'/n42openc_00'+str(yy)+'10101_00'+str(yy+1)+'01231_1M_icemod.nc')
    file_closed = xr.open_mfdataset(inputpath_closed+'/n42tm21_00'+str(yy)+'10101_00'+str(yy+1)+'01231_1M_icemod.nc')
    file_open_list.append(file_open)
    file_closed_list.append(file_closed)

file_ice_open_all = xr.concat(file_open_list, dim='time_counter')
file_ice_closed_all = xr.concat(file_closed_list, dim='time_counter')


In [26]:
file_ice_open_15 = file_ice_open_all['siconc'] > 0.15
file_ice_closed_15 = file_ice_closed_all['siconc'] > 0.15

In [31]:
lon = file_ice_open_all.nav_lon
lat = file_ice_open_all.nav_lat

mask_Arc = (lat >= 50) 
mask_Ant = (lat <= -50) 

In [32]:
sie_Ant_open = (file_ice_open_15.where(mask_Ant) * cellarea_open.isel(deptht=0).drop('deptht')).sum(['x','y']).rename({'time_counter':'time'}).load()
sie_Ant_closed = (file_ice_closed_15.where(mask_Ant) * cellarea_open.isel(deptht=0).drop('deptht')).sum(['x','y']).rename({'time_counter':'time'}).load()
sie_Arc_open = (file_ice_open_15.where(mask_Arc) * cellarea_open.isel(deptht=0).drop('deptht')).sum(['x','y']).rename({'time_counter':'time'}).load()
sie_Arc_closed = (file_ice_closed_15.where(mask_Arc) * cellarea_open.isel(deptht=0).drop('deptht')).sum(['x','y']).rename({'time_counter':'time'}).load()

In [33]:
var_to_plot['mar_sie_arc'] = xr.concat([sie_Arc_open.where(sie_Arc_open['time.month'] == 3, drop=True).squeeze().assign_coords({'cav':'open'}),
                                        sie_Arc_closed.where(sie_Arc_closed['time.month'] == 3, drop=True).squeeze().assign_coords({'cav':'closed'})], dim='cav').assign_coords({'time': var_to_plot.time})/10**12
var_to_plot['sep_sie_ant'] = xr.concat([sie_Ant_open.where(sie_Ant_open['time.month'] == 9, drop=True).squeeze().assign_coords({'cav':'open'}),
                                        sie_Ant_closed.where(sie_Ant_closed['time.month'] == 9, drop=True).squeeze().assign_coords({'cav':'closed'})], dim='cav').assign_coords({'time': var_to_plot.time})/10**12
var_to_plot['sep_sie_arc'] = xr.concat([sie_Arc_open.where(sie_Arc_open['time.month'] == 9, drop=True).squeeze().assign_coords({'cav':'open'}),
                                        sie_Arc_closed.where(sie_Arc_closed['time.month'] == 9, drop=True).squeeze().assign_coords({'cav':'closed'})], dim='cav').assign_coords({'time': var_to_plot.time})/10**12
var_to_plot['feb_sie_ant'] = xr.concat([sie_Ant_open.where(sie_Ant_open['time.month'] == 2, drop=True).squeeze().assign_coords({'cav':'open'}),
                                        sie_Ant_closed.where(sie_Ant_closed['time.month'] == 2, drop=True).squeeze().assign_coords({'cav':'closed'})], dim='cav').assign_coords({'time': var_to_plot.time})/10**12


In [34]:
lon = file_T_open_all.nav_lon
lat = file_T_open_all.nav_lat

mask_regions = xr.Dataset()
mask_regions['AMU'] = (lon >= -109.64) & (lon <= -102.23) & (lat >= -75.80) & (lat <= -71.66)
mask_regions['WROSS'] = (lon >= 157.100) & (lon <= 173.333) & (lat >= -78.130) & (lat <= -74.040)
mask_regions['EROSS'] = (lon >= -176.790) & (lon <= -157.820) & (lat >= -78.870) & (lat <= -77.520)
mask_regions['WWED'] = (lon >= -65.130) & (lon <= -53.020) & (lat >= -75.950) & (lat <= -72.340)
mask_regions['EWED'] = (lon >= -45.647) & (lon <= -32.253) & (lat >= -78.632) & (lat <= -76.899)

In [ ]:
### Bottom properties

mask_cells = (file_T_closed_all['so'] > 0).isel(time=0).drop('time').drop('time_centered').load()
bottom_open = mask_cells.sum('deptht') - 1

mask_cells = (file_T_closed_all['so'] > 0).isel(time=0).drop('time').drop('time_centered').load()
bottom_closed = mask_cells.sum('deptht') - 1

for reg in ['AMU','WROSS','EROSS','WWED','EWED']:    
    print(reg)
    
    var_to_plot['Tbot_'+reg] = xr.concat([file_T_closed_all['thetao'].isel(deptht=bottom_closed).where(mask_regions[reg], drop=True).mean(['x','y']).assign_coords({'cav': 'closed'}),
                                           file_T_open_all['thetao'].isel(deptht=bottom_open).where(mask_regions[reg], drop=True).mean(['x','y']).assign_coords({'cav': 'open'})], dim='cav')
    
    var_to_plot['Sbot_'+reg] = xr.concat([file_T_closed_all['so'].isel(deptht=bottom_closed).where(mask_regions[reg], drop=True).mean(['x','y']).assign_coords({'cav': 'closed'}),
                                           file_T_open_all['so'].isel(deptht=bottom_open).where(mask_regions[reg], drop=True).mean(['x','y']).assign_coords({'cav': 'open'})], dim='cav')

In [147]:
plt.figure()
file_T_open_all['thetao'].isel(deptht=bottom_closed).where(mask_regions[reg], drop=True).isel(time=0).plot()

In [183]:
var_to_plot = var_to_plot.load()

In [191]:
f = plt.figure()
f.set_size_inches(8.25*2, 8.25*2)

ax={}

leg_hdl = []

i = 0


for vv in var_list:
    
    ax[i] = f.add_subplot(5,5,i+1)

    if vv in list(var_to_plot.keys()):
        ax[i].plot(var_to_plot[vv].sel(cav='open'), color='deepskyblue')
        ax[i].plot(var_to_plot[vv].sel(cav='closed'), color='royalblue')
    
    ax[i].axhline(y=var_obs_mean.sel(var=vv), color='black', linewidth=2)
    ax[i].fill_between(x=np.arange(0,100),y1=var_obs_mean.sel(var=vv)-var_obs_std.sel(var=vv), y2=var_obs_mean.sel(var=vv)+var_obs_std.sel(var=vv), color='grey',alpha=0.2)

    ax[i].set_title(vv)

    i = i+1
#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path + 'VALSO_comparison_plot.pdf')


In [ ]:
T_grid_vars_closed['thetao'].isel(deptht=bottom_closed).isel(time_counter=0).plot()

In [ ]:
uocetr_Ross_open = u_open['uocetr_eff'].where(uf.in_range(u_open.nav_lat,[-72.650,-61.600]) & ((u_open.nav_lon <= -135.75) | (u_open.nav_lon >= 360-168.500)), drop=True)


In [ ]:
uocetr_Ross_open.nav_lat.max()

CHECK THAT I AM LOOKING AT THE RIGHT REGIONS FOR THE GYRES

In [ ]:
ref = uocetr_Ross_open.isel(time_counter=0)

lon = ref.nav_lon
lat = ref.nav_lat

proj = ccrs.SouthPolarStereo(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, -90, -50], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [ ]:
ref = cellarea_closed.where(mask_land_closed > 0)

lon = ref.nav_lon
lat = ref.nav_lat

proj = ccrs.SouthPolarStereo(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, -90, -20], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [ ]:
ref = cellarea_open.max('deptht')
plt.figure()

lon = ref.nav_lon
lat = ref.nav_lat

proj = ccrs.SouthPolarStereo(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, -90, -50], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [ ]:
ref = cellarea_closed.where(mask_land_closed > 0)

lon = ref.nav_lon
lat = ref.nav_lat

proj = ccrs.NorthPolarStereo(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, 20, 90], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [ ]:
ref = cellarea_open.isel(deptht=0)
plt.figure()

lon = ref.nav_lon
lat = ref.nav_lat

proj = ccrs.NorthPolarStereo(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, 20, 90], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [141]:
ref = mask_regions['WROSS']
plt.figure()

lon = ref.nav_lon
lat = ref.nav_lat

proj = ccrs.SouthPolarStereo(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, -90, -50], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [ ]:
u_open['uocetr_eff'].sel(x=range(220,221),y=range(79,107)).isel(time_counter=0)

In [ ]:

ref = T_grid_vars_closed_mean10['thetao'].isel(deptht=0).where(mask_land_closed > 0)

lon = ref.nav_lon
lat = ref.nav_lat

proj = ccrs.PlateCarree(central_longitude=0.0, globe=None)
#wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

#theta = np.linspace(0, 2*np.pi, 100)
#center, radius = [0.5, 0.5], 0.5
#verts = np.vstack([np.sin(theta), np.cos(theta)]).T
#circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(lon,lat,ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
#ax1.set_extent([-180, 180, -90, -50], crs=ccrs.PlateCarree())
#ax1.set_boundary(circle, transform=ax1.transAxes)

In [ ]:
T_grid_vars_closed_mean10['so'].isel(deptht=0).plot()

In [ ]:
mask_land_closed = T_grid_vars_closed_mean10['so'].isel(deptht=0)

In [ ]:
mask_land_closed.plot()

In [ ]:
cellarea_closed.where(mask_land_closed)